In [1]:
import pandas as pd

In [2]:
pd.options.display.max_columns = None
data = pd.read_csv("clean_batak_scraped_pdfs.csv")
data.head()

,pdf_path,md_extraction_result,extracted_meaningful_text_v2,n_tokens
0,NaN,# ALIH KODE DAN CAMPUR KODE PADA INTERAKSI JUA...,SKRIPSI\n\n$\sigma(\tau) = \eta$\nDISUSUN OLEH...,13989
1,NaN,# ALIH KODE DAN CAMPUR KODE DALAM KUMPULAN CER...,ALIH KODE DAN CAMPUR KODE DALAM KUMPULAN CERIT...,20762
2,NaN,### ANALISIS TOKOH DALAM NOVEL INONG KARYA RAD...,SKRIPSI\n\nDISUSUN OLEH: NAMA : TRISNA ERAWANI...,7332
3,NaN,# ANALISIS SOSIOLOGI SASTRA TERHADAP CERITA RA...,DIKERJAKAN\n\nUNIVERSITAS SUMATERA UTARA FAKUL...,12362
4,NaN,## ANALISIS PERISTIWA TUTUR DALAM UPACARA ADAT...,ANALISIS PERISTIWA TUTUR DALAM UPACARA ADAT SA...,12039


In [3]:
data.shape
print("Number of columns: {}".format(data.shape[1]))
print("Number of rows: {}".format(data.shape[0]))

Number of columns: 4
Number of rows: 47


In [4]:
data = data.drop(['pdf_path', 'extracted_meaningful_text_v2', 'n_tokens'], axis=1)

In [5]:
data.shape
print("Number of columns: {}".format(data.shape[1]))
print("Number of rows: {}".format(data.shape[0]))

Number of columns: 1
Number of rows: 47


In [6]:
data.isnull().sum()

,0
md_extraction_result,0


In [7]:
data.head()

,md_extraction_result
0,# ALIH KODE DAN CAMPUR KODE PADA INTERAKSI JUA...
1,# ALIH KODE DAN CAMPUR KODE DALAM KUMPULAN CER...
2,### ANALISIS TOKOH DALAM NOVEL INONG KARYA RAD...
3,# ANALISIS SOSIOLOGI SASTRA TERHADAP CERITA RA...
4,## ANALISIS PERISTIWA TUTUR DALAM UPACARA ADAT...


In [8]:
data.duplicated().sum()
data = data.drop_duplicates()

In [9]:
data.shape
print("Number of columns: {}".format(data.shape[1]))
print("Number of rows: {}".format(data.shape[0]))

Number of columns: 1
Number of rows: 47


In [10]:
data['text'] = data['md_extraction_result']
data_to_save = data[['text']]
data_to_save.to_json('extract_batak.jsonl', orient='records', lines=True)

In [11]:
!pip install datatrove tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 94.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 4.9 MB/s eta 0:00:00


**Modified Gopher Quality**

In [12]:
import os
import numpy as np
from datatrove.data import Document
from datatrove.pipeline.filters.base_filter import BaseFilter
from datatrove.pipeline.writers.disk_base import DiskWriter
from datatrove.utils.text import PUNCTUATION_SET


def load_stopwords() -> list[str]:
    """Load stopwords from stopwords-id.txt in current working directory."""
    file_path = "stopwords-id.txt"
    if not os.path.exists(file_path):
        raise FileNotFoundError("File stopwords-id.txt tidak ditemukan!")

    with open(file_path, "r", encoding="utf-8") as f:
        return [line.strip() for line in f if line.strip()]



class GopherQualityFilter(BaseFilter):
    name = "🥇 Gopher Quality"
    _requires_dependencies = ["nltk"]

    def __init__(
        self,
        min_doc_words: int | None = 50,
        max_doc_words: int | None = 100000,
        min_avg_word_length: int | None = 3,
        max_avg_word_length: int | None = 10,
        max_symbol_word_ratio: float | None = 0.1,
        max_bullet_lines_ratio: float | None = 0.9,
        max_ellipsis_lines_ratio: float | None = 0.3,
        max_non_alpha_words_ratio: float | None = 0.8,
        min_stop_words: int | None = 2,
        stop_words: list[str] | None = None,
        exclusion_writer: DiskWriter = None,
    ):
        """
        Filter to apply Gopher's quality heuristic rules.
        Reference: https://arxiv.org/pdf/2112.11446.pdf

        Args:
            min_doc_words:
            max_doc_words:
            min_avg_word_length:
            max_avg_word_length:
            max_symbol_word_ratio:
            max_bullet_lines_ratio:
            max_ellipsis_lines_ratio:
            max_non_alpha_words_ratio:
            min_stop_words:
            stop_words:
            exclusion_writer:
        """
        super().__init__(exclusion_writer)
        self.min_doc_words = min_doc_words
        self.max_doc_words = max_doc_words
        self.min_avg_word_length = min_avg_word_length
        self.max_avg_word_length = max_avg_word_length
        self.max_symbol_word_ratio = max_symbol_word_ratio
        self.max_bullet_lines_ratio = max_bullet_lines_ratio
        self.max_ellipsis_lines_ratio = max_ellipsis_lines_ratio
        self.max_non_alpha_words_ratio = max_non_alpha_words_ratio
        self.min_stop_words = min_stop_words

        # Load stopwords
        self.stop_words = set(load_stopwords())

    def filter(self, doc: Document) -> bool | tuple[bool, str]:
        """

        Args:
            doc: Applies the heuristics rules to decide if a document should be REMOVED


        Returns: False if sample.text does not pass any of the the heuristic tests

        """
        from nltk.tokenize import word_tokenize

        text = doc.text
        words = word_tokenize(text)  # TODO we should use language id filter
        n_words = len(words)

        non_symbol_words = [w for w in words if any(ch not in PUNCTUATION_SET for ch in w)]
        n_non_symbol_words_words = len(non_symbol_words)

        # words < min_doc_words or words > max_doc_words
        if self.min_doc_words and n_non_symbol_words_words < self.min_doc_words:
            return False, "gopher_short_doc"
        if self.max_doc_words and n_non_symbol_words_words > self.max_doc_words:
            return False, "gopher_long_doc"

        # mean word length is outside the range of 3 to 10 characters
        avg_n_words = np.mean([len(w) for w in non_symbol_words])
        if self.min_avg_word_length and avg_n_words < self.min_avg_word_length:
            return False, "gopher_below_avg_threshold"
        if self.max_avg_word_length and avg_n_words > self.max_avg_word_length:
            return False, "gopher_above_avg_threshold"

        # symbol-to-word ratio greater than 0.1 for either the hash symbol or the ellipsis
        if self.max_symbol_word_ratio and text.count("#") / n_words > self.max_symbol_word_ratio:
            return False, "gopher_too_many_hashes"
        if self.max_symbol_word_ratio and (text.count("...") + text.count("…")) / n_words > self.max_symbol_word_ratio:
            return False, "gopher_too_many_ellipsis"

        # any document with more than 90 % of lines starting with a bullet point,
        # or more than 30 % ending with an ellipsis.
        lines = text.splitlines()
        if (
            self.max_bullet_lines_ratio
            and sum(s.lstrip().startswith("•") or s.lstrip().startswith("-") for s in lines) / len(lines)
            > self.max_bullet_lines_ratio
        ):
            return False, "gopher_too_many_bullets"
        if (
            self.max_ellipsis_lines_ratio
            and sum(s.rstrip().endswith("...") or s.rstrip().endswith("…") for s in lines) / len(lines)
            > self.max_ellipsis_lines_ratio
        ):
            return False, "gopher_too_many_end_ellipsis"

        # that 80 % of words in a document contain at least one alphabetic character
        if (
            self.max_non_alpha_words_ratio
            and sum([any((c.isalpha() for c in w)) for w in words]) / n_words < self.max_non_alpha_words_ratio
        ):
            return False, "gopher_below_alpha_threshold"

         # stop word filter
        if self.min_stop_words and sum(w in self.stop_words for w in words) < self.min_stop_words:
            return False, "gopher_enough_stop_words"

        return True


**Modified FineWeb Quality Filter**

In [13]:
"""
The rule that sentences must end with punctuation marks such as “.”, “'”, “"”, “!”, and “?” has been removed in this modification.
"""

from datatrove.pipeline.filters.base_filter import BaseFilter
from datatrove.pipeline.filters.gopher_repetition_filter import find_duplicates

class FineWebQualityFilter(BaseFilter):
    name = "🍷 FineWeb Quality"
    _requires_dependencies = ["nltk"]

    def __init__(
        self,
        exclusion_writer,
        short_line_thr: float = 0.67,
        short_line_length: int = 30,
        char_duplicates_ratio: float = 0.01,
        new_line_ratio: float = 0.3,
    ):
        super().__init__(exclusion_writer)
        self.short_line_threshold = short_line_thr
        self.short_line_length = short_line_length
        self.char_duplicates_ratio = char_duplicates_ratio
        self.new_line_ratio = new_line_ratio

    def filter(self, doc) -> bool | tuple[bool, str]:
        from nltk import word_tokenize


        lines = doc.text.split("\n")
        total_lines = max(1, len(lines))

        # 1) Short line ratio
        ratio = sum(1 for line in lines if len(line) <= self.short_line_length) / total_lines
        if ratio >= self.short_line_threshold:
            return False, "short_line_ratio"

        # 2) Char duplicates ratio
        non_empty_lines = [line for line in lines if line.strip() != ""]
        dup_chars = find_duplicates(non_empty_lines)[1]
        total_chars_no_nl = len(doc.text.replace("\n", ""))
        ratio = (dup_chars / total_chars_no_nl) if total_chars_no_nl > 0 else 0.0
        if ratio >= self.char_duplicates_ratio:
            return False, "char_dup_ratio"

        # 3) New line ratio per token
        words = word_tokenize(doc.text)
        word_count = max(1, len(words))
        new_line = doc.text.count("\n")
        if (new_line / word_count) > self.new_line_ratio:
            return False, "list_ratio"

        return True

In [14]:
import os
import json
from tqdm import tqdm

# Datatrove essentials
from datatrove.executor import LocalPipelineExecutor
from datatrove.pipeline.readers import JsonlReader
from datatrove.pipeline.writers.jsonl import JsonlWriter

# Filters
from datatrove.pipeline.filters import (
    C4QualityFilter,
    GopherRepetitionFilter
)
from datatrove.pipeline.formatters import PIIFormatter

# Deduplication
from datatrove.pipeline.dedup import (
    MinhashDedupSignature,
    MinhashDedupBuckets,
    MinhashDedupCluster,
    MinhashDedupFilter,
)
from datatrove.pipeline.dedup.minhash import MinhashConfig
from datatrove.utils.hashing import HashConfig

# Token counting (optional)
from datatrove.pipeline.tokens import TokensCounter

import nltk
nltk.download('punkt')
nltk.download('punkt_tab')


# ============ Folder Setup ============
folders = [
    "filter_data/logs",
    "filter_data/eliminated/GopherRepetitionFilter",
    "filter_data/eliminated/GopherQualityFilter",
    "filter_data/eliminated/C4QualityFilter",
    "filter_data/eliminated/FineWebQualityFilter",
    "filter_data/minhash/signatures",
    "filter_data/minhash/buckets",
    "filter_data/minhash/remove_ids",
    "filter_data/minhash/final_output",
]
for folder in folders:
    os.makedirs(folder, exist_ok=True)

# ======== Input File ========
INPUT_FILE = "/content/extract_batak.jsonl"

# ======== Minhash Config ========
minhash_config = MinhashConfig(
    hash_config=HashConfig(hash_fc="sha1", precision=64),
    num_buckets=14,
    hashes_per_bucket=8,
    n_grams=5,
)

# ======== Writers untuk simpan doc tereliminasi per filter ========

gopher_rep_writer = JsonlWriter("filter_data/eliminated/GopherRepetitionFilter")
gopher_quality_writer = JsonlWriter("filter_data/eliminated/GopherQualityFilter")
c4_quality_writer = JsonlWriter("filter_data/eliminated/C4QualityFilter")
fineweb_quality_writer = JsonlWriter("filter_data/eliminated/FineWebQualityFilter")

# ======== Stage 0: Filtering ========
filtering_executor = LocalPipelineExecutor(
    pipeline=[
        JsonlReader(INPUT_FILE),
        GopherRepetitionFilter(exclusion_writer=gopher_rep_writer),
        GopherQualityFilter(exclusion_writer=gopher_quality_writer),
        C4QualityFilter( exclusion_writer=c4_quality_writer, filter_no_terminal_punct=False),
        FineWebQualityFilter(exclusion_writer=fineweb_quality_writer),
        JsonlWriter("filter_data/filtered_output")
    ],
    logging_dir="filter_data/logs/filtering"
)

# ======== Jalankan Pipeline ========
filtering_executor.run()
print("✅ Pipeline selesai — hasil disimpan di filter_data/filtered_output")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
2025-12-02 08:01:33.515 | INFO     | datatrove.utils.logging:add_task_logger:58 - Launching pipeline for rank=0
2025-12-02 08:01:33.527 | INFO     | datatrove.utils.logging:log_pipeline:90 - 
--- 🛠️ PIPELINE 🛠
📖 - READER: 🐿 Jsonl
🔻 - FILTER: 👯 Gopher Repetition
🔻 - FILTER: 🥇 Gopher Quality
🔻 - FILTER: ⛰ C4 Quality
🔻 - FILTER: 🍷 FineWeb Quality
💽 - WRITER: 🐿 Jsonl
2025-12-02 08:01:33.543 | INFO     | datatrove.pipeline.readers.base:read_files_shard:201 - Reading input file , 1/1
2025-12-02 08:01:54.963 | SUCCESS  | datatrove.executor.base:_run_for_rank:98 - Processing done for rank=0
2025-12-02 08:01:54.968 | INFO     | datatrove.executor.base:_run_for_rank:104 - 

📉📉📉 Stats: Task 0 📉📉📉

Total Runtime: 21.19 seconds

📖 - READER: 🐿 Jsonl
    Runtime: (0.03%) 0.01 seconds 

✅ Pipeline selesai — hasil disimpan di filter_data/filtered_output


In [15]:
import os
import gzip
import shutil
import glob

# Daftar semua folder writer
folders = [
    "/content/filter_data/filtered_output",
    "/content/filter_data/eliminated/GopherRepetitionFilter",
    "/content/filter_data/eliminated/GopherQualityFilter",
    "/content/filter_data/eliminated/C4QualityFilter",
    "/content/filter_data/eliminated/FineWebQualityFilter",
]

# Fungsi unzip semua .jsonl.gz di folder menjadi .jsonl
def unzip_folder(folder_path):
    if not os.path.isdir(folder_path):
        print(f"{folder_path} bukan folder, dilewati.")
        return

    gz_files = sorted(glob.glob(os.path.join(folder_path, "*.jsonl.gz")))
    if not gz_files:
        print(f"Tidak ada file .jsonl.gz di {folder_path}")
        return

    for gz_file in gz_files:
        output_file = gz_file.rstrip(".gz")  # hasil .jsonl
        with gzip.open(gz_file, 'rb') as f_in:
            with open(output_file, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
        print(f"File berhasil di-unzip: {gz_file} → {output_file}")

# Jalankan untuk semua folder
for folder in folders:
    unzip_folder(folder)


File berhasil di-unzip: /content/filter_data/filtered_output/00000.jsonl.gz → /content/filter_data/filtered_output/00000.jsonl
File berhasil di-unzip: /content/filter_data/eliminated/GopherRepetitionFilter/00000.jsonl.gz → /content/filter_data/eliminated/GopherRepetitionFilter/00000.jsonl
File berhasil di-unzip: /content/filter_data/eliminated/GopherQualityFilter/00000.jsonl.gz → /content/filter_data/eliminated/GopherQualityFilter/00000.jsonl
File berhasil di-unzip: /content/filter_data/eliminated/C4QualityFilter/00000.jsonl.gz → /content/filter_data/eliminated/C4QualityFilter/00000.jsonl
Tidak ada file .jsonl.gz di /content/filter_data/eliminated/FineWebQualityFilter


In [16]:


# ===== Step 1: Minhash Signature =====
stage1_executor = LocalPipelineExecutor(
    pipeline=[
        JsonlReader("/content/filter_data/filtered_output/00000.jsonl"),
        MinhashDedupSignature(
            output_folder="filter_data/minhash/signatures",
            config=minhash_config
        )
    ],
    logging_dir="filter_data/logs/minhash/signatures",

)
stage1_executor.run()

# ===== Step 2: Minhash Buckets =====
stage2_executor = LocalPipelineExecutor(
    pipeline=[
        MinhashDedupBuckets(
            input_folder="filter_data/minhash/signatures",
            output_folder="filter_data/minhash/buckets",
            config=minhash_config
        )
    ],
    logging_dir="filter_data/logs/minhash/buckets",
    tasks=minhash_config.num_buckets,

)
stage2_executor.run()

# ===== Step 3: Minhash Clustering =====
stage3_executor = LocalPipelineExecutor(
    pipeline=[
        MinhashDedupCluster(
            input_folder="filter_data/minhash/buckets",
            output_folder="filter_data/minhash/remove_ids",
            config=minhash_config
        )
    ],
    logging_dir="filter_data/logs/minhash/clustering",

)
stage3_executor.run()

# ===== Step 4: Apply MinhashDedupFilter + Final Output =====
final_executor = LocalPipelineExecutor(
    pipeline=[
        JsonlReader("/content/filter_data/filtered_output/00000.jsonl"),
        TokensCounter(),
        MinhashDedupFilter(input_folder="filter_data/minhash/remove_ids"),
        PIIFormatter(),
        JsonlWriter("filter_data/minhash/final_output/deduped_output")
    ],
    logging_dir="filter_data/logs/minhash/final_output",

)
final_executor.run()

print("Pipeline lokal selesai. Hasil akhir ada di filter_data/minhash/final_output/deduped_output")

2025-12-02 08:01:55.247 | INFO     | datatrove.utils.logging:add_task_logger:58 - Launching pipeline for rank=0
2025-12-02 08:01:55.250 | INFO     | datatrove.utils.logging:log_pipeline:90 - 
--- 🛠️ PIPELINE 🛠
📖 - READER: 🐿 Jsonl
🫂 - DEDUP: 🎯 MinHash stage 1
2025-12-02 08:01:55.255 | INFO     | datatrove.pipeline.readers.base:read_files_shard:201 - Reading input file , 1/1
2025-12-02 08:01:55.776 | INFO     | datatrove.pipeline.dedup.minhash:run:262 - Sorting buckets...
2025-12-02 08:01:55.783 | SUCCESS  | datatrove.executor.base:_run_for_rank:98 - Processing done for rank=0
2025-12-02 08:01:55.786 | INFO     | datatrove.executor.base:_run_for_rank:104 - 

📉📉📉 Stats: Task 0 📉📉📉

Total Runtime: 0.53 seconds

📖 - READER: 🐿 Jsonl
    Runtime: (0.12%) 0 seconds [0.16 milliseconds±0.06 milliseconds/doc]
    Stats: {input_files: 1, doc_len: 143174 [min=19714, max=49276, 35793.50±15485/doc], documents: 4 [4.00/input_file]}
🫂 - DEDUP: 🎯 MinHash stage 1
    Runtime: (99.88%) 0.53 seconds [532.0

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

2025-12-02 08:02:06.450 | SUCCESS  | datatrove.executor.base:_run_for_rank:98 - Processing done for rank=0
2025-12-02 08:02:06.456 | INFO     | datatrove.executor.base:_run_for_rank:104 - 

📉📉📉 Stats: Task 0 📉📉📉

Total Runtime: 1.16 seconds

📖 - READER: 🐿 Jsonl
    Runtime: (0.10%) 0 seconds [0.29 milliseconds±0.10 milliseconds/doc]
    Stats: {input_files: 1, doc_len: 143174 [min=19714, max=49276, 35793.50±15485/doc], documents: 4 [4.00/input_file]}
🔢 - TOKENIZER: 📊 Counter
    Runtime: (96.82%) 1.12 seconds [1 second and 124.40 milliseconds±0 milliseconds/batch]
    Stats: {tokens: 57083 [min=7669, max=19850, 14270.75±6459/doc]}
🫂 - DEDUP: 🎯 MinHash stage 4
    Stats: {total: 4, forwarded: 4}
✂️ - FORMAT: 📞 PII
    Runtime: (1.76%) 0.02 seconds [5.11 milliseconds±2.26 milliseconds/doc]
    Stats: {total: 4}
💽 - WRITER: 🐿 Jsonl
    Runtime: (1.32%) 0.02 seconds [3.83 milliseconds±4.03 milliseconds/doc]
    Stats: {XXXXX.jsonl.gz: 4, total: 4, doc_len: 143174 [min=19714, max=49276, 357

Pipeline lokal selesai. Hasil akhir ada di filter_data/minhash/final_output/deduped_output


In [17]:
import gzip
import shutil


file_path = "/content/filter_data/minhash/final_output/deduped_output/00000.jsonl.gz"
output_path = "/content/filter_data/minhash/final_output/deduped_output/00000.jsonl"


with gzip.open(file_path, "rb") as f_in:
    with open(output_path, "wb") as f_out:
        shutil.copyfileobj(f_in, f_out)

print("✅ File berhasil diekstrak ke:", output_path)


✅ File berhasil diekstrak ke: /content/filter_data/minhash/final_output/deduped_output/00000.jsonl


In [18]:
import pandas as pd

# daftar file jsonl yang ingin dikonversi
files = {
    "extract_batak": "/content/extract_batak.jsonl",
    "filtered_output" : "/content/filter_data/filtered_output/00000.jsonl",
    "GopherRepetition_eliminated": "/content/filter_data/eliminated/GopherRepetitionFilter/00000.jsonl",
    "GopherQuality_eliminated": "/content/filter_data/eliminated/GopherQualityFilter/00000.jsonl",
    "C4Quality_eliminated": "/content/filter_data/eliminated/C4QualityFilter/00000.jsonl",
    "FineWebQuality_eliminated": "/content/filter_data/eliminated/FineWebQualityFilter/00000.jsonl",
    "deduped_output": "/content/filter_data/minhash/final_output/deduped_output/00000.jsonl"
}

output_excel = "/content/no_basic_cleaning_output.xlsx"

# tulis ke excel
with pd.ExcelWriter(output_excel, engine="openpyxl") as writer:
    for sheet_name, file_path in files.items():
        try:
            print(f"Membaca: {file_path}")
            df = pd.read_json(file_path, lines=True)
            df.to_excel(writer, sheet_name=sheet_name[:31], index=False)
            #  Excel membatasi nama sheet max 31 karakter
        except Exception as e:
            print(f"Gagal membaca {file_path}: {e}")

print(f"\nSelesai! File Excel tersimpan di: {output_excel}")


Membaca: /content/extract_batak.jsonl
Membaca: /content/filter_data/filtered_output/00000.jsonl
Membaca: /content/filter_data/eliminated/GopherRepetitionFilter/00000.jsonl
Membaca: /content/filter_data/eliminated/GopherQualityFilter/00000.jsonl
Membaca: /content/filter_data/eliminated/C4QualityFilter/00000.jsonl
Membaca: /content/filter_data/eliminated/FineWebQualityFilter/00000.jsonl
Gagal membaca /content/filter_data/eliminated/FineWebQualityFilter/00000.jsonl: Expected object or value
Membaca: /content/filter_data/minhash/final_output/deduped_output/00000.jsonl


/tmp/ipython-input-570050850.py:21: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(file_path, lines=True)



Selesai! File Excel tersimpan di: /content/no_basic_cleaning_output.xlsx
